In this notebook, we will load the LLLM GeneZC/MiniChat-3B to see how it performs. Here are the main steps:

Step 1. Transform user's input to vector with Sentence Transform.

Step 2. Compute the similarity between vectorized users' input and vectorized "movie recommendation": 

    - If the similarity is high, we then response based on model's knowledge and then perform vector search based on additional information from the response.

    - Otherwise, we will let the model answer with its own knowledge.

We note that suitable prompts are very important to extract useful information and generate meaningful answer.

In [98]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

In [99]:
# Load the model 
MODEL_NAME = 'google/flan-t5-small'

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Define tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [100]:
print(tokenizer.pad_token)

</s>


In [101]:
# Load sentence transformer model
from sentence_transformers import SentenceTransformer
filter_model = SentenceTransformer('all-MiniLM-L6-v2')

In [102]:
# Get vector from an input text by Sentence Transformer
def get_vector(text):
    text_vector = filter_model.encode(text)
    return torch.tensor(text_vector)

In [103]:
# Vectorization of "movie recommendation"
valid_query = "movie recommendation"
valid_vector = get_vector(valid_query)
valid_vector.shape

torch.Size([384])

In [104]:
# Generate response based on a suitable prompt
def bot_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    # Generate answer
    output_sequences = model.generate(
        **inputs,
        max_length=1000,
        num_return_sequences=1,
        temperature=0.1,  # Adjust for creativity
        top_p=0.1,  # Nucleus sampling
        top_k=50,  # Top-k sampling
        no_repeat_ngram_size=2  # Prevent repeating n-grams
    )
    
    # Process the output to remove the input question from the response if it gets included
    answer = tokenizer.decode(output_sequences[0], skip_special_tokens=True).replace(prompt, "").strip()
    return answer

In [105]:
# Connect to the Qdrant database
from qdrant_client import models, QdrantClient
from qdrant_client.http.models import CollectionDescription, VectorParams

client = QdrantClient(host='localhost', port=6333)

In [106]:
# Perform vector database search
def database_search(input_text):
    results = []
    hits = client.search(
        collection_name='movies',
        query_vector=filter_model.encode(input_text).tolist(),
        limit = 2
    )
    for idx, hit in enumerate(hits):
        result = {}
        result['original_title'] = hit.payload['original_title']
        result['genres'] = hit.payload['genres']
        result['overview'] = hit.payload['overview']
        results.append(result)
    return results

In [107]:
def answer_with_query(text, results):
    # Constructing a detailed prompt that structures the results as individual pieces of evidence
    evidence_list = "\n".join([f"Title: {result['original_title']}\nGenres: {result['genres']}\nOverview: {result['overview']}" for result in results])
    # prompt = f"Based on the following information, answer the question '{text}':\n\n{evidence_list}\n\nAnswer:"
    
    # answer = bot_response(prompt)
    return evidence_list

In [108]:
intentions = [
    "Can you recommend to me a movie similar to Harry Potter?",
    "What's the weather like today?",
    "Book a table for two at a Mexican restaurant tonight.",
    "Tell me a joke.",
    "How can I reset my password?",
    "Play some music from the 90s.",
    "What was the highest-grossing film in 2020?",
    "Find me a flight to New York next weekend.",
    "I need a recipe for a chocolate cake.",
    "What's the latest news on technology?",
    "Show me my photos from Paris.",
    "How do I make a Manhattan cocktail?",
    "Suggest a good workout for abs.",
    "What are some vegan restaurants nearby?",
    "Is there a good comedy movie to watch?",
    "I want to learn Python programming.",
    "Where can I watch The Office?",
    "Give me directions to the nearest gas station.",
    "What are the symptoms of COVID-19?",
    "Tell me about the life of Albert Einstein.",
    "Show me the latest horror films.",
    "What animated movies are recommended for kids?",
    "List all movies starring Tom Hanks.",
    "Who directed Pulp Fiction?",
    "Find movies similar to La La Land.",
    "What's the plot of The Matrix?",
    "Give me the soundtrack list from Guardians of the Galaxy.",
    "How many Oscars did Forrest Gump win?",
    "Is 'Joker' suitable for teenagers?",
    "What are some must-watch classic films?",
    "Play the trailer for 'Avengers: Infinity War'.",
    "I'm in the mood for a comedy. Any suggestions?",
    "What films are based on Stephen King's books?",
    "When was Schindler's List released?",
    "Find me a movie that will make me cry.",
    "What are the best sci-fi movies of the last decade?",
    "Which movies are in the Marvel Cinematic Universe?",
    "Tell me about the directorial style of Wes Anderson.",
    "How do I access movies on Netflix?",
    "Rate 'Mad Max: Fury Road' out of 10."
]

In [109]:
from torch.nn.functional import cosine_similarity

for intention in intentions:
    intention_vector = get_vector(intention)
    similarity = cosine_similarity(intention_vector.unsqueeze(0), valid_vector.unsqueeze(0))
    print(f"Question: {intention}")
    if similarity < 0.5:
        prompt = f"Answer the following question:\n{intention}"
        print(bot_response(prompt))
    if similarity >= 0.5:
        prompt = f"For the information mentioned, describe its genres and any notable themes:\n{intention}"
        bot_answer = bot_response(prompt)
        print(f"First answer:\n{bot_answer}")
        results = database_search(bot_answer)
        evidence_list = answer_with_query(intention, results)
        print(f"Databse search results:\n{evidence_list}")
        # print(f"Final answer:\n{answer}")
    print("============================================================")
        

Question: Can you recommend to me a movie similar to Harry Potter?


/Users/khanhdam/.local/share/virtualenvs/movie-recommendation-LLM-K7t7D6Dz/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/khanhdam/.local/share/virtualenvs/movie-recommendation-LLM-K7t7D6Dz/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


First answer:
The Harry Potter series is a great adventure film.
Databse search results:
Title: King Kong
Genres: Action, Adventure, Drama, Horror
Overview: An adventure film about a film crew in search of a monster on a remote island. The crew finds King Kong and decides to take him back to New York as a money making spectacle. The film is a masterpiece of Stop-Motion in filmmaking history and inspired a line of King Kong films.
Title: Disaster!
Genres: Action, Animation, Comedy, Science Fiction
Overview: A spoof of disaster films, an asteroid is coming towards earth and Harry Bottoms is in charge of saving us all...again...
Question: What's the weather like today?
a sunny day
Question: Book a table for two at a Mexican restaurant tonight.
I'm not sure if I can afford it. I think I will be able to afford the food.
Question: Tell me a joke.
a snarky taylor
Question: How can I reset my password?
Use a swab of the password to reset it.
Question: Play some music from the 90s.
i love a lot

By this small test, we can see in the model answer well in general, however, there are a few questions the model does not response well. We would need to fine-tune it with the dataset wiki_movies from hugging face.